In [1]:
class Parent():
    def __init__(self, a=1, b=2, c=3):
        self.a=a
        self.b=b
        self.c=c
    def _print(self, s):
        print(f"I`m parent: a={self.a}\tb={self.b}\tc={self.c}\n{s}")
    
    def __call__(self, s):
        self._print(s)

class Child(Parent):
    def _print(self, s):
        print(f"I`m child: a={self.a}\tb={self.b}\tc={self.c}\n{s}")


In [2]:
child = Child(a=11)
child("woca")

I`m child: a=11	b=2	c=3
woca
